## Orien Questions

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

output = "output2.xlsx"

In [ ]:

# -------------------------------
# 1. Clean helper functions
# -------------------------------
def normalize_list(x):
    """Convert a cell into a clean Python list of strings."""
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [str(i).strip() for i in x]
    x = str(x).replace("[", "").replace("]", "").replace("'", "")
    return [i.strip() for i in x.split(",") if i.strip()]

def precision_recall_f1(gt_list, pred_list):
    """
    Compute micro-precision, micro-recall, micro-F1 for multi-label sets.
    """
    all_labels = sorted(list(set().union(*gt_list, *pred_list)))

    # Convert lists to multi-hot vectors
    y_true = [[1 if label in gt else 0 for label in all_labels] for gt in gt_list]
    y_pred = [[1 if label in pr else 0 for label in all_labels] for pr in pred_list]

    precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="micro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    return precision, recall, f1


In [ ]:
# -------------------------------
# 2. Load Excel
# -------------------------------
df = pd.read_excel(output, sheet_name=0)

# Normalize list-like fields
df["EB Codes"] = df["EB Codes"].apply(normalize_list)
df["Extracted EB Codes"] = df["Extracted EB Codes"].apply(normalize_list)
df["Phrases"] = df["Phrases"].apply(normalize_list)
df["Extracted Canonical Term"] = df["Extracted Canonical Term"].apply(normalize_list)

# -------------------------------
# 3. EB CODE METRICS
# -------------------------------
eb_prec, eb_rec, eb_f1 = precision_recall_f1(
    df["EB Codes"], df["Extracted EB Codes"]
)

# -------------------------------
# 4. PHRASE METRICS
# -------------------------------
ph_prec, ph_rec, ph_f1 = precision_recall_f1(
    df["Phrases"], df["Extracted Canonical Term"]
)

# -------------------------------
# 5. COMBINED METRIC (PAIR EB + PHRASE)
# Treat each pair as one label.
# -------------------------------
df["combined_gt"] = df.apply(
    lambda row: [f"{eb}__{ph}" for eb in row["EB Codes"] for ph in row["Phrases"]],
    axis=1
)

df["combined_pred"] = df.apply(
    lambda row: [f"{eb}__{ph}" for eb in row["Extracted EB Codes"] for ph in row["Extracted Canonical Term"]],
    axis=1
)

comb_prec, comb_rec, comb_f1 = precision_recall_f1(
    df["combined_gt"], df["combined_pred"]
)

df


,Question,EB Codes,Phrases,Response,Query ID,Extracted EB Codes,Extracted Phrases,Extracted Canonical Term,TTFOT,TTFRT,Total Reason Tokens,Total Output Tokens,Total Time for Reason,Total Time for Output,Total Time (LLM),Total Time (API),combined_gt,combined_pred
0,What is my copay for an urgent care visit?,"[EB01, EB03]","[Co-Payment, Urgent Care]","{\n ""eb_codes"": [""EB01"", ""EB03""],\n ""phrases...",2025-12-05-28996,"[EB01, EB03]","copay, urgent care visit","[Co-Payment, Urgent Care]",8.547408,0.094980,456,84,8.452422,1.558132,10.105551,14.828270,"[EB01__Co-Payment, EB01__Urgent Care, EB03__Co...","[EB01__Co-Payment, EB01__Urgent Care, EB03__Co..."
1,Do I need to pay anything if the urgent care c...,"[EB12, EB03]","[Urgent Care, Out-Of-Plan-Network]","```json\n{\n ""eb_codes"": [""EB01"", ""EB03"", ""EB...",2025-12-05-28997,"[EB01, EB03, EB12]","pay anything, urgent care center, out of network","[Co-Payment, Emergency Services, Out-Of-Plan-N...",34.770446,0.090578,1874,126,34.679864,2.389255,37.159713,41.785097,"[EB12__Urgent Care, EB12__Out-Of-Plan-Network,...","[EB01__Co-Payment, EB01__Emergency Services, E..."
2,I had a sprained ankle and went to urgent care...,[EB03],"[Urgent Care, Diagnostic X-Ray]","```json\n{\n ""eb_codes"": [""EB03"", ""EB06""],\n ...",2025-12-05-28998,"[EB03, EB06]","x‑ray, urgent care, last week","[Diagnostic X-Ray, Urgent Care, Week]",24.619600,0.096811,1324,99,24.522784,1.876702,26.496314,31.334046,"[EB03__Urgent Care, EB03__Diagnostic X-Ray]","[EB03__Diagnostic X-Ray, EB03__Urgent Care, EB..."
3,I received a bill for urgent care that says it...,"[EB01, EB03]","[Non-Covered, Urgent Care]","{\n ""eb_codes"": [""EB01"", ""EB03""],\n ""phrases...",2025-12-05-28999,"[EB01, EB03]","not covered, urgent care","[Non-Covered, Urgent Care]",7.703259,0.091637,408,84,7.611618,1.570452,9.273723,14.059533,"[EB01__Non-Covered, EB01__Urgent Care, EB03__N...","[EB01__Non-Covered, EB01__Urgent Care, EB03__N..."
4,Is urgent care covered if I’m out of state?,"[EB01, EB03]","[Active Coverage, Urgent Care]","{\n ""eb_codes"": [""EB03"", ""EB12""],\n ""phrases...",2025-12-05-29000,"[EB03, EB12]","urgent care, out of state","[Urgent Care, Out-Of-Plan-Network]",22.148911,0.090110,1187,82,22.058797,1.544298,23.693220,28.770041,"[EB01__Active Coverage, EB01__Urgent Care, EB0...","[EB03__Urgent Care, EB03__Out-Of-Plan-Network,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Will pediatric services count toward my deduct...,"[EB01, EB03]","[Deductible, Pediatric]","{\n ""eb_codes"": [""EB01"", ""EB03""],\n ""phrases...",2025-12-05-29219,"[EB01, EB03]","deductible, pediatric services","[Deductible, Pediatric]",8.035852,0.086602,423,78,7.949246,1.467302,9.503165,14.335427,"[EB01__Deductible, EB01__Pediatric, EB03__Dedu...","[EB01__Deductible, EB01__Pediatric, EB03__Dedu..."
224,I need a pediatric test but it’s not covered. ...,"[EB01, EB03]","[Non-Covered, Pediatric]","{\n ""eb_codes"": [""EB01"", ""EB03""],\n ""phrases...",2025-12-05-29220,"[EB01, EB03]","not covered, pediatric test","[Non-Covered, Pediatric]",10.352761,0.088335,548,84,10.264421,1.580914,11.933687,16.179420,"[EB01__Non-Covered, EB01__Pediatric, EB03__Non...","[EB01__Non-Covered, EB01__Pediatric, EB03__Non..."
225,"If I’m out of state, are pediatric services co...","[EB01, EB03]","[Active Coverage, Pediatric]","{\n ""eb_codes"": [""EB03"", ""EB12""],\n ""phrases...",2025-12-05-29221,"[EB03, EB12]","pediatric services, out of state","[Pediatric, Out-Of-Plan-Network]",35.548116,0.090798,1892,88,35.457314,1.678699,37.226828,41.984470,"[EB01__Active Coverage, EB01__Pediatric, EB03_...","[EB03__Pediatric, EB03__Out-Of-Plan-Network, E..."
226,Will I pay anything if the pediatric provider ...,"[EB12, EB03]","[Pediatric, Out-Of-Plan-Network]","{\n ""eb_codes"": [""EB12"", ""EB03""],\n ""phrases...",2025-12-05-29222,"[EB12, EB03]","out‑of‑network, pediatric provider","[Out-Of-Plan-Network, Pediatric]",17.201584,0.090443,915,90,17.

In [ ]:


# -------------------------------
# 6. PRINT RESULTS
# -------------------------------
print("\n=== EB CODE METRICS ===")
print("Precision:", round(eb_prec, 4))
print("Recall:", round(eb_rec, 4))
print("F1:", round(eb_f1, 4))

print("\n=== PHRASE METRICS ===")
print("Precision:", round(ph_prec, 4))
print("Recall:", round(ph_rec, 4))
print("F1:", round(ph_f1, 4))

print("\n=== COMBINED (EB + PHRASE) METRICS ===")
print("Precision:", round(comb_prec, 4))
print("Recall:", round(comb_rec, 4))
print("F1:", round(comb_f1, 4))


=== EB CODE METRICS ===
Precision: 0.8911
Recall: 0.8717
F1: 0.8813

=== PHRASE METRICS ===
Precision: 0.7687
Recall: 0.8123
F1: 0.7899

=== COMBINED (EB + PHRASE) METRICS ===
Precision: 0.6781
Recall: 0.7516
F1: 0.713


In [28]:
#  -------------------------------
# 6. Store metrics in DataFrame
# -------------------------------
metrics_df = pd.DataFrame({
    "Metric": ["EB Codes", "Cannonical Terms", "Combined (EB + Cannonical Terms)"],
    "Precision": [eb_prec, ph_prec, comb_prec],
    "Recall": [eb_rec, ph_rec, comb_rec],
    "F1 Score": [eb_f1, ph_f1, comb_f1]
})

# Round for display clarity
metrics_df = metrics_df.round(4)


# -------------------------------
# 7. Save to output.xlsx (sheet: metrics)
# -------------------------------
with pd.ExcelWriter("output1.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    metrics_df.to_excel(writer, sheet_name="Metrics", index=False)

print("Metrics saved successfully to output.xlsx (sheet: Metrics)")

Metrics saved successfully to output.xlsx (sheet: Metrics)


## Old Data

In [46]:
path = "output2.xlsx"

In [51]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# -------------------------------
# 1. Clean helper functions
# -------------------------------
def normalize_list(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [str(i).strip() for i in x]
    x = str(x).replace("[", "").replace("]", "").replace("'", "")
    return [i.strip() for i in x.split(",") if i.strip()]

def precision_recall_f1(gt_list, pred_list):
    all_labels = sorted(list(set().union(*gt_list, *pred_list)))
    y_true = [[1 if label in gt else 0 for label in all_labels] for gt in gt_list]
    y_pred = [[1 if label in pr else 0 for label in all_labels] for pr in pred_list]
    precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="micro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)
    return precision, recall, f1


# -------------------------------
# 2. Load Excel
# -------------------------------
df = pd.read_excel(path)

# Normalize lists
df["EB Codes"] = df["EB Codes"].apply(normalize_list)
df["Extracted EB Codes"] = df["Extracted EB Codes"].apply(normalize_list)
df["Phrases"] = df["Phrases"].apply(normalize_list)
df["Extracted Canonical Term"] = df["Extracted Canonical Term"].apply(normalize_list)


# -------------------------------
# 3. FILTER PREDICTIONS PAIRWISE
# -------------------------------
TARGET_EB = {"EB01", "EB03", "EB12"}

def filter_pairs(eb_list, phrase_list):
    filtered_eb = []
    filtered_phrases = []

    for eb, ph in zip(eb_list, phrase_list):
        if eb in TARGET_EB:
            filtered_eb.append(eb)
            filtered_phrases.append(ph)

    return filtered_eb, filtered_phrases

df["Extracted EB Codes"], df["Extracted Canonical Term"] = zip(*df.apply(
    lambda row: filter_pairs(row["Extracted EB Codes"], row["Extracted Canonical Term"]),
    axis=1
))

# Sort both GT & predicted data to avoid ordering problems
df["Phrases"] = df["Phrases"].apply(lambda lst: sorted(lst))
df["Extracted Canonical Term"] = df["Extracted Canonical Term"].apply(lambda lst: sorted(lst))

df["EB Codes"] = df["EB Codes"].apply(lambda lst: sorted(lst))
df["Extracted EB Codes"] = df["Extracted EB Codes"].apply(lambda lst: sorted(lst))

# -------------------------------
# 4. EB METRICS
# -------------------------------
eb_prec, eb_rec, eb_f1 = precision_recall_f1(
    df["EB Codes"], df["Extracted EB Codes"]
)

# -------------------------------
# 5. PHRASE METRICS
# -------------------------------
ph_prec, ph_rec, ph_f1 = precision_recall_f1(
    df["Phrases"], df["Extracted Canonical Term"]
)

df


,Question,EB Codes,Phrases,Response,Query ID,Extracted EB Codes,Extracted Phrases,Extracted Canonical Term,TTFOT,TTFRT,Total Reason Tokens,Total Output Tokens,Total Time for Reason,Total Time for Output,Total Time (LLM),Total Time (API)
0,What is my copay for an urgent care visit?,"[EB01, EB03]","[Co-Payment, Urgent Care]","{\n ""eb_codes"": [""EB01"", ""EB06""],\n ""phrases...",2025-12-05-29224,[EB01],"copay, visit",[Co-Payment],13.861840,0.499954,721,74,13.361881,1.385362,15.247212,20.031892
1,Do I need to pay anything if the urgent care c...,"[EB03, EB12]","[Out-Of-Plan-Network, Urgent Care]","{\n ""eb_codes"": [""EB01"", ""EB12""],\n ""phrases...",2025-12-05-29225,"[EB01, EB12]","pay anything, out of network","[Co-Payment, Out-Of-Plan-Network]",29.489969,0.098751,1596,88,29.391214,1.648142,31.138123,35.738697
2,I had a sprained ankle and went to urgent care...,[EB03],"[Urgent Care, X-Ray]","{\n ""eb_codes"": [""EB03"", ""EB06""],\n ""phrases...",2025-12-05-29226,[EB03],"x‑ray, urgent care, week",[X-Ray],20.631018,0.102574,1112,93,20.528440,1.736249,22.367280,26.765247
3,I received a bill for urgent care that says it...,"[EB01, EB03]","[Non-Covered, Urgent Care]","```json\n{\n ""eb_codes"": [""EB01"", ""EB03""],\n ...",2025-12-05-29227,"[EB01, EB03]","not covered, urgent care","[Non-Covered, Urgent Care]",7.986530,0.090147,425,88,7.896378,1.652894,9.639436,14.089302
4,Is urgent care covered if I’m out of state?,"[EB01, EB03]","[Active, Urgent Care]","{\n ""eb_codes"": [""EB03""],\n ""phrases"": {\n ...",2025-12-05-29228,[EB03],urgent care,[Urgent Care],13.327945,0.094492,715,49,13.233448,0.921314,14.249271,18.600525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Will pediatric services count toward my deduct...,"[EB01, EB03]","[Deductible, Pediatric]","{\n ""eb_codes"": [""EB01"", ""EB03""],\n ""phrases...",2025-12-05-29447,"[EB01, EB03]","deductible, pediatric services","[Deductible, Pediatric]",8.031434,0.085806,424,84,7.945619,1.574523,9.605970,14.089285
224,I need a pediatric test but it’s not covered. ...,"[EB01, EB03]","[Non-Covered, Pediatric]","{\n ""eb_codes"": [""EB01"", ""EB03""],\n ""phrases...",2025-12-05-29448,"[EB01, EB03]","not covered, pediatric test","[Non-Covered, Pediatric]",11.049999,0.088189,587,84,10.961804,1.576423,12.626434,17.134987
225,"If I’m out of state, are pediatric services co...","[EB01, EB03]","[Active, Pediatric]","{\n ""eb_codes"": [""EB03""],\n ""phrases"": {\n ...",2025-12-05-29449,[EB03],pediatric services,[Pediatric],13.930997,0.085918,742,49,13.845074,0.929275,14.860282,19.390594
226,Will I pay anything if the pediatric provider ...,"[EB03, EB12]","[Out-Of-Plan-Network, Pediatric]","{\n ""eb_codes"": [""EB01"", ""EB12""],\n ""phrases...",2025-12-05-29450,"[EB01, EB12]","pay anything, out-of-network","[Co-Insurance, Co-Payment]",25.596355,0.088917,1368,93,25.507434,1.756894,27.353260,31.810804


In [48]:
print("\n=== EB CODE METRICS ===")
print("Precision:", round(eb_prec, 4))
print("Recall:", round(eb_rec, 4))
print("F1:", round(eb_f1, 4))

print("\n=== PHRASE METRICS ===")
print("Precision:", round(ph_prec, 4))
print("Recall:", round(ph_rec, 4))
print("F1:", round(ph_f1, 4))


=== EB CODE METRICS ===
Precision: 0.9282
Recall: 0.845
F1: 0.8847

=== PHRASE METRICS ===
Precision: 0.7766
Recall: 0.721
F1: 0.7478


In [49]:
#  -------------------------------
# 6. Store metrics in DataFrame
# -------------------------------
metrics_df = pd.DataFrame({
    "Metric": ["EB Codes", "Cannonical Terms"],
    "Precision": [eb_prec, ph_prec],
    "Recall": [eb_rec, ph_rec],
    "F1 Score": [eb_f1, ph_f1]
})

# Round for display clarity
metrics_df = metrics_df.round(4)
metrics_df

,Metric,Precision,Recall,F1 Score
0,EB Codes,0.9282,0.845,0.8847
1,Cannonical Terms,0.7766,0.721,0.7478


In [50]:
with pd.ExcelWriter(path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="Output", index=False)

# -------------------------------
# 7. Save to output.xlsx (sheet: metrics)
# -------------------------------
with pd.ExcelWriter(path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    metrics_df.to_excel(writer, sheet_name="Metrics", index=False)

print(f"Metrics saved successfully to {path} (sheet: Metrics)")

Metrics saved successfully to output2.xlsx (sheet: Metrics)
